In [1]:
import torch

In [2]:
print(torch.__version__.split('+'))

['1.8.0', 'cu101']


In [3]:
!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-{torch.__version__.split('+')[0]}+cu101.html
!pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-{torch.__version__.split('+')[0]}+cu101.html
!pip install torch-cluster -f https://pytorch-geometric.com/whl/torch-{torch.__version__.split('+')[0]}+cu101.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{torch.__version__.split('+')[0]}+cu101.html
!pip install torch-geometric
# !pip install networkx

Looking in links: https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
     |████████████████████████████████| 2.6MB 7.0MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
     |████████████████████████████████| 1.5MB 7.3MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
     |████████████████████████████████| 1.0MB 6.1MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
     |████████████████████████████████| 389kB 7.9MB/s 
     |████████████████████████████████| 194kB 8.7MB/s 
     |████████████████████████████████| 235kB 14.6MB/s 
     |████████████████████████████████| 2.2MB 17.3MB/s 
     |████████████████████████████████| 51kB 6.9MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-1.6.3-cp37-none-any.whl size=322719 sha256=faac7afca1e3b93ebb938125eacbc982cd70e7e111f9e0e8007d5d4c98f85212
  Stored in directory: /root/.cache/pip/wheels/6d/47/1e/0af8ce3e21783c3e584c22502011a3367

# 讀取資料

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# *資料創建*
x: 特徵向量
y: 類別
edge_index: data format 

Initial feature
dim : (num_nodes, 3)

In [5]:
from torch_geometric.data import Data, DataLoader
from torch_geometric.utils import degree
import warnings
warnings.filterwarnings("ignore")

## 隨機圖

In [6]:
def syn_fetcher(n= 5000):
  syn_data_list = list()
  for i in range(30):
    with open(f'/content/drive/MyDrive/python_data/社群網路與推薦系統/hw1/Synthetic/{n}/{i}.txt', 'r') as f:
      with open(f'/content/drive/MyDrive/python_data/社群網路與推薦系統/hw1/Synthetic/{n}/{i}_score.txt', 'r') as f_score:
        edge_list = [line.strip().split('\t') for line in f.readlines()]
        edge_list = [[int(edge) for edge in line] for line in edge_list]
        edge_index = torch.tensor(edge_list, dtype=torch.long)
        in_nodes = torch.tensor([edge[0] for edge in edge_list])
        out_nodes = torch.tensor([edge[1] for edge in edge_list])
        num_nodes = n
        # data = Data(edge_index=edge_index.t().contiguous())
        # in_nodes, out_nodes = data['edge_index']
        nodes_degree = degree(in_nodes, num_nodes= num_nodes, dtype= torch.float) + degree(out_nodes, num_nodes= num_nodes, dtype= torch.float)
        Xv = torch.tensor([[dv.item(), 1, 1] for dv in nodes_degree])
        Yv = [float(line.strip().split('\t')[1]) for line in f_score.readlines()]
        data = Data(x= Xv, y= Yv, edge_index=edge_index.t().contiguous())
        syn_data_list.append(data)
  print('done!')
  torch.cuda.empty_cache()
  return syn_data_list

In [7]:
# syn_data_list = syn_fetcher()

## Real

In [8]:
def real_fetcher(name= 'com-youtube'):
  real_data_list = list() 
  with open(f'/content/drive/MyDrive/python_data/社群網路與推薦系統/hw1/Real/{name}.txt', 'r') as f:
    with open(f'/content/drive/MyDrive/python_data/社群網路與推薦系統/hw1/Real/{name}_score.txt', 'r') as f_score:
      edge_list = [line.strip().split(' ') for line in f.readlines()]
      edge_list = [[int(edge) for edge in line] for line in edge_list]
      edge_index = torch.tensor(edge_list, dtype=torch.long)
      in_nodes = torch.tensor([edge[0] for edge in edge_list])
      out_nodes = torch.tensor([edge[1] for edge in edge_list])
      num_nodes = max(torch.max(in_nodes).item(), torch.max(out_nodes).item()) + 1
      print(f'num nodes: {num_nodes}')
      # data = Data(edge_index=edge_index.t().contiguous())
      # in_nodes, out_nodes = data['edge_index']
      nodes_degree = degree(in_nodes, num_nodes= num_nodes, dtype= torch.float) + degree(out_nodes, num_nodes= num_nodes, dtype= torch.float)
      Xv = torch.tensor([[dv.item(), 1, 1] for dv in nodes_degree])
      Yv = [float(line.strip().split('\t')[1]) for line in f_score.readlines()]
      data = Data(x= Xv, y= Yv, edge_index=edge_index.t().contiguous())
      real_data_list.append(data)

  return real_data_list

In [9]:
from torch_geometric.nn import MessagePassing
import torch.nn as nn 

# DrBC

1. The user only has to define the functions 𝜙 , i.e. message(), and 𝛾 , i.e. update(), as well as the aggregation scheme to use, i.e. aggr="add", aggr="mean" or aggr="max".

2. tensors passed to propagate() can be mapped to the respective nodes 𝑖 and 𝑗 by appending _i or _j to the variable name, .e.g. x_i and x_j. Note that we generally refer to 𝑖 as the central nodes that aggregates information, and refer to 𝑗 as the neighboring nodes

3. The normalization coefficients are derived by the node degrees deg(𝑖) for each node 𝑖 which gets transformed to 1/(deg(𝑖)‾‾‾‾‾‾√⋅deg(𝑗)‾‾‾‾‾‾√) for each edge (𝑗,𝑖)∈. The result is saved in the tensor norm of shape [num_edges, ] (step 3).

## 模型



In [10]:
class DrBC(MessagePassing):
  def __init__(self, out_channels= 128, num_layers= 5, init_out_channels= 3, aggr= 'add'):
    super(DrBC,self).__init__(aggr= aggr)
    self.out_channels= out_channels
    self.num_layers= num_layers
    self.W0 = nn.Linear(init_out_channels, out_channels)
    self.W4 = nn.Linear(out_channels, out_channels)
    self.W5 = nn.Linear(out_channels, 1)
    self.GRU = torch.nn.GRUCell(input_size= out_channels, hidden_size= out_channels, bias= True)

  def forward(self, x, edge_index):
    '''x: [num_nodes, init_dim] edge_index: [2, num_edges]'''

    # Step2: 線性變換
    first_embedding = torch.nn.ReLU()(self.W0(x))
    '''Normalization'''
    v_norm = torch.norm(first_embedding, dim= 1).view(-1,1)
    first_embedding = first_embedding/v_norm

    # Step3: 對變換後節點特徵進行標準化
    in_nodes, out_nodes = edge_index
    in_degree = degree(in_nodes, num_nodes= x.size(0), dtype= torch.float) 
    out_degree = degree(out_nodes, num_nodes= x.size(0), dtype= torch.float)
    deg = in_degree + out_degree 
    deg_inv_sqrt = (deg+1).pow(-0.5)
    norm = deg_inv_sqrt[in_nodes] * deg_inv_sqrt[out_nodes]

    for i in range(self.num_layers):
      if i == 0:
        print('first layer message passing')
        message = self.propagate(edge_index, x=first_embedding, norm= norm)
        x = self.GRU(message, first_embedding) # [N, out_channels]
        '''TODO Normalize x matrix by row'''
        v_norm = torch.norm(x, dim= 1).view(-1,1)
        x = x/v_norm
        h = x.unsqueeze(dim= 1) # [N, 1, out_channels]
      else:
        '''釋放記憶體'''
        torch.cuda.empty_cache()
        print(f'{i} layer message passing')
        message = self.propagate(edge_index, x=x, norm= norm)
        x = self.GRU(message, x)
        v_norm = torch.norm(x, dim= 1).view(-1,1)
        x = x/v_norm
        h = torch.cat([h, x.unsqueeze(dim= 1)], dim= 1)
    
    '''釋放記憶體'''
    torch.cuda.empty_cache()
    # z, _ = torch.max(h, dim= 1) # max_pooling layers
    z = nn.AdaptiveAvgPool2d((1, self.out_channels))(h)
    z = z.squeeze(dim= 1)
    y = self.W4(z)
    y = torch.nn.ReLU()(y)
    y = self.W5(y)

    return y
  # self.propagate(edge_index, x=x, norm=norm) from MassagePassing 調用 message函數
  def message(self, x_j, norm): 
    # x_j has shape [E, out_channels]
    return norm.view(-1, 1) * x_j

  def update(self,aggr_out):
    return aggr_out

  def pair_wise_topk(self, pred, target, k= 0.05):
    # args: pred [V, 1] target [V, 1]
    size = pred.size()[0]
    pred_topk = torch.topk(pred, k= int(size*k), dim= 0).indices
    tgt_topk = torch.topk(target, k= int(size*k), dim= 0).indices

    return len(np.intersect1d(pred_topk.cpu(), tgt_topk.cpu()))/(size*k)
  
  def pair_kendall_tau(self, pred, target):

    pred_rank = stats.rankdata(pred.flatten().data.tolist())
    tgt_rank = stats.rankdata(target.flatten().data.tolist())
    tau, p = stats.kendalltau(x= pred_rank, y= tgt_rank)

    return tau

# 訓練

1. 將所有資料Load進來
2. random sample 5|V| nodes to form 5|V| node pairs 

In [11]:
from torch.optim import Adam
from torch.nn import BCEWithLogitsLoss, Sigmoid
import numpy as np 
import scipy.stats as stats
import networkx as nx
import time 

In [12]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print(f'using {device}')

# device = torch.device('cuda')

using cuda


## 參數設定

In [13]:
out_channels = 64
lr= 1e-4
batch_size= 8
node_sampling = 5
iteration = 160
L = 5
n = 2000
k = 0.01

In [14]:
# model = DrBC(out_channels= out_channels, num_layers= L, init_out_channels= 3, aggr= 'add').to(device)
# optimizer = Adam(model.parameters(), lr= lr)
# criterion = BCEWithLogitsLoss(reduction= 'sum')
# for name, param in model.named_parameters():
#     if param.requires_grad:
#         print(name) #param.data)

## function

In [15]:
def test(model, data_list= None, k= 0.01):

  model.eval()
  topk_list = []
  tau_list = []
  time_list = []

  for data in data_list:

    with torch.no_grad():
      st = time.time()
      print('starting prediction...')
      preds = model(data['x'].to(device), data['edge_index'].to(device))
      print('prediction ends...')
      en = time.time()
      labels = torch.tensor(data['y'], dtype= torch.float).view(-1,1).to(device)

      topk_list.append(model.pair_wise_topk(pred= preds, target= labels, k= k))
      tau_list.append(model.pair_kendall_tau(pred= preds, target= labels))
      time_list.append(round(en-st,2))
  
  print('-'*50)
  print(f'time: {round(en-st,2)}')
  print(f'Mean time: {round(np.mean(time_list), 3)}')
  print(f'std time: {round(np.std(time_list), 3)}')
  print(f'Mean topk: {round(np.mean(topk_list), 3)}')
  print(f'std topk: {round(np.std(topk_list), 3)}')
  print(f'Mean tau: {round(np.mean(tau_list), 3)}')
  print(f'std tau: {round(np.std(tau_list), 3)}')
  print(f'test done!!!!')
  print('-'*50)

In [16]:
def train(model, n):

  model.train()
  topk_list = []
  tau_list = []

  for epoch in range(iteration):

    if ((epoch+1) % batch_size) == 0:
      print(f'{epoch+1}th epoch...')
    
    '''生成訓練資料'''

    G = nx.generators.random_graphs.powerlaw_cluster_graph(n= n, m=4 ,p= 0.05)
    # nodes = [x for x in G.nodes()]
    edge_list = [x for x in G.edges()]
    in_nodes = torch.tensor([x[0] for x in G.edges()])
    out_nodes = torch.tensor([x[1] for x in G.edges()])
    edge_index = torch.tensor(edge_list, dtype=torch.long)

    nodes_degree = degree(in_nodes, num_nodes= n, dtype= torch.float) + degree(out_nodes, num_nodes= n, dtype= torch.float)
    Xv = torch.tensor([[dv.item(), 1, 1] for dv in nodes_degree])
    st = time.time()
    Yv = list(nx.betweenness_centrality(G, k= int(0.5 * Xv.size(0)), normalized=True).values())
    en = time.time()

    data = Data(x= Xv, y= Yv, edge_index=edge_index.t().contiguous())

    # print(f'bc計算: {en-st} s')
    # st = time.time()

    preds = model(data['x'].to(device), data['edge_index'].to(device))
    labels = torch.tensor(data['y'], dtype= torch.float).view(-1,1).to(device)

    '''metrics'''
    topk_list.append(model.pair_wise_topk(pred= preds, target= labels, k= k))
    tau_list.append(model.pair_kendall_tau(pred= preds, target= labels))

    '''sample nodes to calc loss'''
    V = data.num_nodes 
    src_node_ids = set(data['edge_index'][0].data.tolist())
    tgt_node_ids = set(data['edge_index'][1].data.tolist())
    src_sample_ids = np.random.choice(list(src_node_ids), size=5*V, replace=True)
    tgt_sample_ids = np.random.choice(list(tgt_node_ids), size=5*V, replace=True)

    input = preds[src_sample_ids] - preds[tgt_sample_ids]
    target = labels[src_sample_ids] - labels[tgt_sample_ids]

    loss = criterion(input=input, target=Sigmoid()(target))
    loss.backward()

    '''梯度累積'''
    if ((epoch+1) % batch_size) == 0:
      optimizer.step()
      optimizer.zero_grad()
      print(f'Mean topk: {round(np.mean(topk_list),3)}')
      print(f'Mean tau: {round(np.mean(tau_list),3)}')
      topk_list = []
      tau_list = []

      test(model= model, data_list= syn_data_list)
      model.train()

    # if (epoch+1) % 20 == 0:
    #   print(f'Mean topk: {round(np.mean(topk_list),3)}')
    #   print(f'Mean tau: {round(np.mean(tau_list),3)}')
    #   topk_list = []
    #   tau_list = []

    #   test(model= model, data_list= syn_data_list)
    #   test(model= model, data_list= real_data_list)

    #   model.train()
  
  print('training process done!!!')
  return model

# 訓練及測試資料
save 訓練好的model

In [17]:
# trained_model = train(model= model, n= n)
# torch.save(trained_model, f'DrBC_{n}_{out_channels}.pt')
# !cp -f DrBC_{n}_{out_channels}.pt '/content/drive/My Drive/python_data/社群網路與推薦系統/hw1'

## 測試
load 訓練好的model

In [18]:
torch.cuda.empty_cache()

In [19]:
trained_model = torch.load(f'/content/drive/My Drive/python_data/社群網路與推薦系統/hw1/DrBC_{n}_{out_channels}.pt', map_location=torch.device(device))

## Syn data

In [20]:
# for n in [5000, 10000, 20000, 50000, 100000]:
#   for k in [0.01]:
    
#     syn_data_list = syn_fetcher(n= n)
#     test(model= trained_model.to(device), data_list= syn_data_list, k= k)
#     print(f'k: {k}')
#     print(f'n: {n}')

## Real data

In [21]:
name = 'com-youtube'
# name = 'amazon'
# name = 'dblp'
# name = 'com-lj'
real_data_list = real_fetcher(name= name)
test(model= trained_model.to(device), data_list= real_data_list, k= 0.1)

num nodes: 1134890
starting prediction...
first layer message passing
1 layer message passing
2 layer message passing
3 layer message passing
4 layer message passing
prediction ends...
--------------------------------------------------
time: 0.79
Mean time: 0.79
std time: 0.0
Mean topk: 0.588
std topk: 0.0
Mean tau: 0.668
std tau: 0.0
test done!!!!
--------------------------------------------------


In [22]:
data = real_data_list[0]

In [23]:
data

Data(edge_index=[2, 2987624], x=[1134890, 3], y=[1134890])

In [24]:
# model.eval()
# topk_list = []
# tau_list = []
# time_list = []

# for data in data_list:

#   with torch.no_grad():
#     st = time.time()
#     preds = model(data['x'].to(device), data['edge_index'].to(device))
#     en = time.time()
#     labels = torch.tensor(data['y'], dtype= torch.float).view(-1,1).to(device)

#     topk_list.append(model.pair_wise_topk(pred= preds, target= labels, k= k))
#     tau_list.append(model.pair_kendall_tau(pred= preds, target= labels))
#     time_list.append(round(en-st,2))

# print('-'*50)
# print(f'time: {round(en-st,2)}')
# print(f'Mean time: {round(np.mean(time_list), 3)}')
# print(f'std time: {round(np.std(time_list), 3)}')
# print(f'Mean topk: {round(np.mean(topk_list), 3)}')
# print(f'std topk: {round(np.std(topk_list), 3)}')
# print(f'Mean tau: {round(np.mean(tau_list), 3)}')
# print(f'std tau: {round(np.std(tau_list), 3)}')
# print(f'test done!!!!')
# print('-'*50)